# CNOT

In this notebook CNOT gate is introduced.

CNOT is defined as 
- *"flip the target if control is |1>"*, 
- or $target:=control\oplus target$ , 
- or `if control { target := X @ target }`.

Let us derive a matrix for CNOT. 

Truth table is:

```
c t | c T
---------
0 0 | 0 0
0 1 | 0 1
1 0 | 1 1   <---
1 1 | 1 0   <---
```

In [ ]:
import numpy as np

ZERO = np.array([1., 0.]).T
ONE = np.array([0., 1.]).T

In [ ]:
CNOT = np.zeros((4, 4))

# what's happening here?
CNOT[:, 0b00] = np.kron(ZERO, ZERO)
CNOT[:, 0b01] = np.kron(ZERO, ONE)
CNOT[:, 0b10] = np.kron(ONE, ONE)
CNOT[:, 0b11] = np.kron(ONE, ZERO)

In [ ]:
for ic, control in enumerate((ZERO, ONE)):
    for it, target in enumerate((ZERO, ONE)):
        system_state = np.kron(control, target)
        print(f"system |{ic}{it}>: {system_state.T} -> ", end="")
        new_state = CNOT @ system_state
        print(new_state)

In [ ]:
print(CNOT)

# The most important question
How CNOT acts, if control is in a superposition? This shed some light on the entanglement.

In [ ]:
PLUS = (.5 ** .5) * (ZERO + ONE)
MINUS = (.5 ** .5) * (ZERO - ONE)
print("|+> =", PLUS, "; |-> =", MINUS)

In [ ]:
CNOT @ (np.kron(PLUS, ZERO))

In [ ]:
CNOT @ (np.kron(MINUS, ZERO))

In [ ]:
for ic, control in enumerate((ZERO, ONE)):
    for it, target in enumerate((ZERO, ONE)):
        
        # <CT| CNOT (|-> ⊗ |0>)
        # or just <CT|CNOT|-0>
        amplitude = np.kron(control, target).T.conj() @ CNOT @ (np.kron(MINUS, ZERO))
        
        # |<CT|CNOT|-0>|
        magnitute = abs(amplitude)
        
        # |<CT|CNOT|-0>|^2
        probability = magnitute ** 2
        print(f"P(|{ic}{it}>) = | <{ic}{it}|CNOT|-0> |^2 = {probability:.3f}")

In [ ]:
# this is how we measure. Note, that the value on the amplitude 
# correspond to a APRIORI probability, thus, <1.
# APRIORI = P(CONIDITIONAL) * P(CONDITION)
# P(CT) = P(C|T)*P(T)

I_OTIMES_ONE = (np.kron(np.eye(2), ONE.reshape(2, 1)))
I_OTIMES_ONE.T.conj() @ CNOT @ (np.kron(MINUS, ZERO))

# Can we write the same in qiskit?

In [ ]:
from qiskit import BasicAer, execute, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Operator
backend = BasicAer.get_backend("statevector_simulator")

In [ ]:
control = QuantumRegister(1, "control")
target = QuantumRegister(1, "target")
control_bit = ClassicalRegister(1, "control bit")
target_bit = ClassicalRegister(1, "target bit")

qc = QuantumCircuit(target, control, target_bit, control_bit)

# ok, this should be MINUS \otimes ZERO
qc.h(control)
qc.z(control)
display(qc.draw('mpl', scale=.5, reverse_bits=True))
execute(qc, backend).result().get_statevector().real

In [ ]:
op = qc.cx(control, target)
display(qc.draw('mpl', scale=.5, reverse_bits=True))
# qiskit has a reversed bit ordering
# this is ok, but if you want to see matrices, which match with theory
# you will have to agree, that CX is applied other way :)
print(op[0].operation.to_matrix())

In [ ]:
execute(qc, backend).result().get_statevector().real

In [ ]:
qc.measure(target, target_bit)
display(qc.draw('mpl', scale=.5, reverse_bits=True))

In [ ]:
# probabilistic outcome, note, that control is not measured, thus bit = 0
for i in range(10):
    r = execute(qc, backend).result()
    print("C =", r.get_statevector().real, " \tT= ", r.get_counts())

In [ ]:
qc.measure(control, control_bit)
display(qc.draw('mpl', scale=.5, reverse_bits=True))

In [ ]:
# probabilistic outcome
for i in range(10):
    r = execute(qc, backend).result()
    print(r.get_statevector().real, "  \t", r.get_counts())